In [1]:
import sys
import numpy as np
import gensim
import pandas as pd
from gensim.models.doc2vec import Doc2Vec,TaggedDocument





C:\Users\60139\miniconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
movies = pd.read_csv('./data/movie_with_tag.csv')
genres=pd.read_csv('./data/genre_id.csv')

In [3]:
print(movies.shape[0])

27278


In [4]:
movie_list=[]

In [5]:
for i in range(movies.shape[0]):
    movie_dic = {}
    movie_dic['id'] = movies['movieId'][i]
    movie_dic['title'] = movies['title'][i]
    movie_dic['genres'] = movies['genres'][i]
    movie_dic['tags'] = movies['tags'][i]
    movie_dic['genre_vector']=[0 for i in range(19)]
    movie_list.append(movie_dic)

In [6]:
for i in range(len(movie_list)):
    movie_list[i]['tags']=movie_list[i]['tags'].split('|')
    movie_list[i]['title']=movie_list[i]['title'].split(' ')

In [7]:
tag_train = []
for item in movie_list:
    document = TaggedDocument(item['tags'],tags=[int(item['id'])])
    tag_train.append(document)

In [8]:
title_train = []
for item in movie_list:
    document = TaggedDocument(item['title'],tags=[int(item['id'])])
    title_train.append(document)

In [9]:
print(movie_list[0]['genre_vector'])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [10]:
genre_dict={}
for i in range(genres.shape[0]):
    genre_dict[genres['genre'][i]] = i

In [11]:
print(genre_dict)

{'Sci-Fi': 0, 'Horror': 1, 'Comedy': 2, 'Musical': 3, 'Action': 4, 'Animation': 5, 'Mystery': 6, 'War': 7, 'Romance': 8, 'Film-Noir': 9, 'Adventure': 10, 'Crime': 11, 'Fantasy': 12, 'Children': 13, 'Thriller': 14, 'Drama': 15, 'Western': 16, 'IMAX': 17, 'Documentary': 18}


In [12]:
for i in range(len(movie_list)):
    gen_list=movie_list[i]['genres'].split('|')
    for item in gen_list:
        if genre_dict.__contains__(item):
            movie_list[i]['genre_vector'][genre_dict[item]]=1
        

In [13]:
def train(x_train, size=109):
    model = Doc2Vec(x_train,min_count=1,window=3,vector_size=size,sample=1e-3,negative=5,workers=4)
    model.train(x_train,total_examples=model.corpus_count,epochs=10)
    return model


In [14]:
model_tag = train(tag_train,84)
model_title = train(title_train,25)

In [16]:
movieId=[]
movieVec=[]
for item in movie_list:
    movieId.append(item['id'])
    tag=item['tags']
    title=item['title']
    tag_vector = model_tag.infer_vector(doc_words=tag,alpha=0.025,steps=300)
    title_vector = model_title.infer_vector(doc_words=title,alpha=0.025,steps=300)
    movieVec.append(list(title_vector)+item['genre_vector']+list(tag_vector))

In [17]:
dataframe = pd.DataFrame({'movieId': movieId, 'movieVec': movieVec})

In [18]:
dataframe.to_csv("./movie_vec.csv", index=False, sep=',')